In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# Data Preprocessing

In [2]:
dataset_path = "/kaggle/input/fer2013/"

In [3]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "train",
    image_size=(48, 48),
    batch_size=32,
    color_mode="grayscale",
    shuffle=True
)

Found 28709 files belonging to 7 classes.


In [4]:
ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path + "test",
    image_size=(48, 48),
    batch_size=32,
    color_mode="grayscale",
    shuffle=False
)

Found 7178 files belonging to 7 classes.


# Training a classifier

In [5]:
model = tf.keras.Sequential([
    # Block 1
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),

    # Block 2
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),

    # Block 3
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),

    # Block 4
    tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),

    # Fully Connected Layers (Head part)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    
    # Output Layer
    tf.keras.layers.Dense(7, activation='softmax')  # FER-2013 has 7 emotion classes
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
model.fit(ds_train, validation_data=ds_valid, epochs=500, callbacks=[tf.keras.callbacks.EarlyStopping(min_delta=0.001, monitor="val_loss", restore_best_weights=True, patience=20)], verbose=1)

Epoch 1/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 70s 58ms/step - accuracy: 0.2053 - loss: 2.3593 - val_accuracy: 0.3587 - val_loss: 1.6146
Epoch 2/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - accuracy: 0.3791 - loss: 1.6155 - val_accuracy: 0.4299 - val_loss: 1.5126
Epoch 3/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - accuracy: 0.4923 - loss: 1.3292 - val_accuracy: 0.5242 - val_loss: 1.2274
Epoch 4/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 27s 31ms/step - accuracy: 0.5303 - loss: 1.2374 - val_accuracy: 0.5316 - val_loss: 1.2166
Epoch 5/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.5574 - loss: 1.1765 - val_accuracy: 0.5469 - val_loss: 1.2141
Epoch 6/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.5726 - loss: 1.1440 - val_accuracy: 0.5426 - val_loss: 1.1990
Epoch 7/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.5980 - loss: 1.0748 - val_accuracy: 0.5878 - val_loss: 1.1115
Epoch 8/500
898/898 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.6231 - loss: 1

In [8]:
model.save("face_emotion_classifier.keras")